In [499]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid
import ccxt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [500]:
c.OHLCV_COLUMNS

['time_epoch', 'open', 'high', 'low', 'close', 'volume']

### CCXT Client

In [ ]:
print(ccxt.exchanges) # print a list of all available exchange classes

In [674]:
import ccxt
polo = ccxt.poloniex({
    'apiKey': cfg.POLONIEX_API_KEY,
    'secret': cfg.POLONIEX_API_SECRET_KEY,
})
gdax = ccxt.gdax({
    'apiKey': cfg.GDAX_API_KEY,
    'secret': cfg.GDAX_API_SECRET_KEY,
    'password': cfg.GDAX_PASSPHRASE,
    'verbose':False,
})
binance = ccxt.binance()

exchange = binance
markets = exchange.load_markets()

#print(exchange.id, markets)
#print(exchange.fetch_order_book(exchange.symbols[0]))
#print(exchange.fetch_ticker('ETH/BTC')) # coin/market - base/quote
#print(exchange.fetch_trades('LTC/BTC'))

### Ingest Data

In [691]:
import calendar 

def utc_to_epoch(utc_time):
    return calendar.timegm(utc_time.utctimetuple())

def epoch_to_utc(epoch_sec):
    return datetime.datetime.utcfromtimestamp(epoch_sec)

# Prices
def get_price_data_fpath(coin, market, exchange_id, period):
    fname = '{:s}_{:s}_{:s}_{:s}.csv'.format(
        exchange_id, coin, market, str(period))
    return os.path.join(cfg.DATA_DIR, fname)

def get_symbol(coin, market):
    return coin+'/'+market

def fetch_ohlcv_data(exchange, coin, market, period, start=None, end=None):
    """ 
    Start/End = time in UTC 
    """
    print("Downloading:", get_symbol(coin, market))
    assert period in exchange.timeframes
    data = exchange.fetch_ohlcv(get_symbol(coin, market), period)
    df = make_ohlcv_df(data, start, end)
    print("Downloaded rows:", len(df))
    return df

def fetch_and_save_ohlcv_data(exchange, coin, market, period, start, end):
    df = fetch_ohlcv_data(exchange, coin, market, period, start, end)
    fpath = get_price_data_fpath(coin, market, exchange.id, period)
    df.to_csv(fpath, index=True)
    return df

def update_local_ohlcv_data(exchange, coin, market, period, start, end):
    fpath = get_price_data_fpath(coin, market, exchange.id, period)
    if os.path.exists(fpath):
        df = fetch_ohlcv_data(exchange, coin, market, period, start, end)
        df = merge_local_csv_feeds(df, fpath)
    else:
        df = fetch_and_save_ohlcv_data(exchange, coin, market, period, start, end)
    return df

def load_chart_data(exchange_id, coin, market, period):
    fpath = get_price_data_fpath(coin, market, exchange_id, period)
    df = pd.read_csv(fpath, index_col='time_epoch')
    df['time_utc'] = [epoch_to_utc(t) for t in df.index]
    return df

def download_chart_data(exchange, coins, market, period, start, end):
    for coin in coins:
        update_local_ohlcv_data(exchange, coin, market, period, start, end)

def load_currency_group_prices(exchange_id, coins, market, period):
    df = pd.DataFrame()
    for coin in coins:
        symbol = get_symbol(coin, market)
        data = load_chart_data(exchange_id, coin, market, period)
        df[symbol] = data['close']
    df.dropna(inplace=True)
    df['time_utc'] = [epoch_to_utc(t) for t in df.index]
    return df

def get_time_range(df, start_utc=None, end_utc=None):
    if start_utc is not None:
        df = df[df.index >= utc_to_epoch(start_utc)]
    if end_utc is not None:
        df = df[df.index < utc_to_epoch(end_utc)]
    return df

def make_ohlcv_df(data, start=None, end=None):
    df = pd.DataFrame(data, columns=c.OHLCV_COLUMNS)
    df['time_epoch'] = df['time_epoch'] // 1000 # ccxt includes millis
    df['time_utc'] = [epoch_to_utc(t) for t in df['time_epoch']] 
    df.set_index('time_epoch', inplace=True)
    df.sort_index(inplace=True)
    df = get_time_range(df, start, end)
    return df
        
def merge_local_csv_feeds(new_data, fpath):
    cur_df = pd.read_csv(fpath, index_col='time_epoch')
    cur_df['time_utc'] = [epoch_to_utc(t) for t in cur_df.index]
    new_df = pd.DataFrame(new_data)
    cur_df = pd.concat([cur_df, new_df])
    cur_df = cur_df[~cur_df.index.duplicated(keep='last')]
    cur_df.to_csv(fpath, index=True)
    return df

In [692]:
# Get historical data
coins = [c.BTC, c.ETH, c.LTC]#, c.ETH, c.XRP, c.XMR, c.DASH]
market = c.USDT
coin = c.LTC
symbol = get_symbol(coin, market)
period = '30m' #1800
#start = datetime.datetime(year=2018, month=1, day=4, hour=14)
start = datetime.datetime.utcnow() - datetime.timedelta(hours=6)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=2)

In [693]:
df = fetch_ohlcv_data(exchange, coin, market, period, start, end)
df = fetch_and_save_ohlcv_data(exchange, coin, market, period, start, end)
df = load_chart_data(exchange.id, coin, market, period)
download_chart_data(exchange, coins, market, period, start, end)
df = load_currency_group_prices(exchange.id, coins, market, period)
df = update_local_ohlcv_data(exchange, coin, market, period, start, end)

Downloading: LTC/USDT
Downloaded rows: 12
Downloading: LTC/USDT
Downloaded rows: 12
Downloading: BTC/USDT
Downloaded rows: 12
Downloading: ETH/USDT
Downloaded rows: 12
Downloading: LTC/USDT
Downloaded rows: 12
Downloading: LTC/USDT
Downloaded rows: 12


### Exchange

In [24]:
exchange.fetch_balance()

{'BCH': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'BTC': {'free': 0.6, 'total': 0.6, 'used': 0.0},
 'ETH': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'LTC': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'USD': {'free': 486.3, 'total': 20951.3, 'used': 20465.0},
 'free': {'BCH': 0.0, 'BTC': 0.6, 'ETH': 0.0, 'LTC': 0.0, 'USD': 486.3},
 'info': [{'available': '0',
   'balance': '0.0000000000000000',
   'currency': 'LTC',
   'hold': '0.0000000000000000',
   'id': 'ab22380c-b2df-4ff0-8902-f74b9336687c',
   'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'},
  {'available': '0',
   'balance': '0.0000000000000000',
   'currency': 'ETH',
   'hold': '0.0000000000000000',
   'id': '2d41385a-80e4-476d-8ea8-93c3c0833a09',
   'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'},
  {'available': '0.6',
   'balance': '0.6000000000000000',
   'currency': 'BTC',
   'hold': '0.0000000000000000',
   'id': '1a758fd9-b6fa-47ae-a004-be8e9602ca50',
   'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'},


In [34]:
from enum import Enum, unique

@unique
class TradeMode(Enum):
    BACKTEST = 0
    SIMULATE = 1
    LIVE = 2

In [149]:
import pandas as pd

class DataFeed():
    def __init__(self, fpath, start=None, end=None):
        self.fpath = fpath
        self.start = start
        self.end = end
        self.prior_time = None
        self.history = None
        self.init()

    def init(self):
        pass
    
    def update(self):
        pass
            
    def next(self):
        self.update()
        assert len(self.history) > 0
        
        if self.prior_time is None:
            row = self.history.iloc[0]
            self.prior_time = row.name
            return row

        data = self.history[self.history.index > self.prior_time]
        if len(data) > 0:
            row = data.iloc[0]
            self.prior_time = row.name
            return row
        else:
            print("No new data after:", self.prior_time)
    
    def __len__(self):
        return len(self.history)

    
class CSVDataFeed(DataFeed):
    def __init__(self, fpath, start, end):
        super().__init__(fpath, start, end)

    def init(self):
        self.update()
    
    def update(self):
        df = pd.read_csv(self.fpath, parse_dates=True, index_col='time')
        df.sort_index(inplace=True)
        if self.start is not None:
            df = df[df.index >= start]            
        if self.end is not None:
            df = df[df.index < end]
        self.history = df
        
    
class ExchangeDataFeed():
    def __init__(self, fpath, start, end, client):
        super().__init__(fpath, start, end)
        self.client = client

    def init(self):
        self.update()
    
    def update(self):
        df = pd.read_csv(self.fpath, parse_dates=True, index_col='time')
        df.sort_index(inplace=True)
        if self.start is not None:
            df = df[df.index >= start]            
        if self.end is not None:
            df = df[df.index < end]
        self.history = df

In [135]:
feed_fpath = os.path.join(cfg.DATA_DIR, 'gdax_BTC_USD_1m.csv')
df = DataFeed(feed_fpath, start=)

In [136]:
len(df)

351

In [117]:
df.history.head()

,open,high,low,close,volume
time,,,,,
2018-01-04 11:10:00,14800.00,14800.89,14800.00,14800.89,0.948353
2018-01-04 11:11:00,14800.89,14800.89,14800.88,14800.89,8.619182
2018-01-04 11:12:00,14800.89,14800.89,14800.87,14800.88,9.053623
2018-01-04 11:13:00,14800.88,14800.88,14800.87,14800.88,8.873541
2018-01-04 11:14:00,14800.88,14850.00,14800.88,14850.00,2.194248


In [138]:
df.history.iloc[0],df.history.iloc[-1]

(open      14800.000000
 high      14800.890000
 low       14800.000000
 close     14800.890000
 volume        0.948353
 Name: 2018-01-04 11:10:00, dtype: float64, open      15053.180000
 high      15053.180000
 low       15053.170000
 close     15053.170000
 volume        3.109679
 Name: 2018-01-04 17:00:00, dtype: float64)

In [151]:
start = datetime.datetime(year=2018, month=1, day=4, hour=16)
end = datetime.datetime.now()
df = DataFeed(feed_fpath, start, end)
row = df.next()
while row is not None:
    print(df.start, df.end, df.prior_time, row)
    row = df.next()

2018-01-04 16:00:00 2018-01-04 17:44:12.757794 2018-01-04 16:00:00 open      15145.000000
high      15176.860000
low       15140.040000
close     15143.870000
volume        7.479868
Name: 2018-01-04 16:00:00, dtype: float64
2018-01-04 16:00:00 2018-01-04 17:44:12.757794 2018-01-04 16:01:00 open      15143.170000
high      15170.000000
low       15140.000000
close     15169.990000
volume        6.742475
Name: 2018-01-04 16:01:00, dtype: float64
2018-01-04 16:00:00 2018-01-04 17:44:12.757794 2018-01-04 16:02:00 open      15170.00000
high      15170.00000
low       15101.11000
close     15146.25000
volume        3.52426
Name: 2018-01-04 16:02:00, dtype: float64
2018-01-04 16:00:00 2018-01-04 17:44:12.757794 2018-01-04 16:03:00 open      15135.340000
high      15135.340000
low       15101.110000
close     15117.550000
volume        8.068068
Name: 2018-01-04 16:03:00, dtype: float64
2018-01-04 16:00:00 2018-01-04 17:44:12.757794 2018-01-04 16:04:00 open      15117.550000
high      15117.550

In [154]:
df.next()

No new data after: 2018-01-04 17:00:00


In [37]:
class Punisher():
    def __init__(self, exchange=None, feed=None, strategy=None, record=None):
        self.exchange = exchange
        self.feed = feed
        self.strategy = strategy
        self.record = record
        
    def run(self, mode=False):
        if mode == TradeMode.BACKTEST:
            # do backtesting
            pass
        elif mode == TradeMode.SIMULATE:
            # place fake trades
            pass
        elif mode == TradeMode.LIVE:
            # trade live
            pass

In [38]:
p = Punisher(None,None,None,None)

In [ ]:
class Record():
    def __init__(self):
        pass
    
    def save(self, data):
        pass
    
    def save_trade(self, order):
        pass

In [39]:
class Strategy():
    def __init__(self, context):
        self.ctx = context
        self.initialize()
    
    def initialize(self):
        self.ctx.symbol = 'BTC/USD'
    
    def handle_data(self, data):
        # Handle next iteration of data from feed
        order = self.ctx.exchange.create_limit_buy_order(
            symbol, amount=1, price=2500.00)
        self.ctx.metrics.save(data.current(self.ctx.symbol, 'price'))
        self.ctx.metrics.save(data.current(self.ctx.symbol, 'volume'))
        self.ctx.metrics.save(order, 'order')
    
    
class TestStrategy():
    def __init__(self):
        pass
    
    def next(self, data):
        pass

In [40]:
s = Strategy()

In [25]:
    
        
class TestExchange():
    def __init__(self):
        self.balance = {}
        self.orderbooks = {}
        self.orders = {}
        self.trades = {}
        self.currencies = {}
        self.symbols = None
        self.precision = {}
        self.limits = {}
        self.fees = {'trading': {}, 'funding': {}}
    
    def fetch_balance(self):
        """
        returns: 
            {'BCH': {'free': 0.0, 'total': 0.0, 'used': 0.0},
             'BTC': {'free': 0.6, 'total': 0.6, 'used': 0.0},
             'free': {'BCH': 0.0, 'BTC': 0.6, 'ETH': 0.0, 'LTC': 0.0, 'USD': 486.3},
             'info': []}
        """
        pass
    
    def fetch_ticker(self, symbol):
        return {
            'symbol': symbol,
            'timestamp': timestamp,
            'datetime': self.iso8601(timestamp),
            'high': None,
            'low': None,
            'bid': bid,
            'ask': ask,
            'vwap': None,
            'open': None,
            'close': None,
            'first': None,
            'last': self.safe_float(ticker, 'price'),
            'change': None,
            'percentage': None,
            'average': None,
            'baseVolume': float(ticker['volume']),
            'quoteVolume': None,
            'info': ticker,
        }
    

In [26]:
exchange.fetch_balance('B)

{'BCH': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'BTC': {'free': 0.6, 'total': 0.6, 'used': 0.0},
 'ETH': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'LTC': {'free': 0.0, 'total': 0.0, 'used': 0.0},
 'USD': {'free': 486.3, 'total': 20951.3, 'used': 20465.0},
 'free': {'BCH': 0.0, 'BTC': 0.6, 'ETH': 0.0, 'LTC': 0.0, 'USD': 486.3},
 'info': [{'available': '0',
   'balance': '0.0000000000000000',
   'currency': 'LTC',
   'hold': '0.0000000000000000',
   'id': 'ab22380c-b2df-4ff0-8902-f74b9336687c',
   'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'},
  {'available': '0',
   'balance': '0.0000000000000000',
   'currency': 'ETH',
   'hold': '0.0000000000000000',
   'id': '2d41385a-80e4-476d-8ea8-93c3c0833a09',
   'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'},
  {'available': '0.6',
   'balance': '0.6000000000000000',
   'currency': 'BTC',
   'hold': '0.0000000000000000',
   'id': '1a758fd9-b6fa-47ae-a004-be8e9602ca50',
   'profile_id': '5226417b-1995-4df8-86ea-ea7ac9c33ec6'},


In [ ]:
EX = Exchange(gdax)

### Trading

In [ ]:
# Trading
print(exchange.fetch_balance())

# sell one ฿ for market price and receive $ right now
print(exchange.id, exchange.create_market_sell_order('BTC/USD', 1))

# limit buy BTC/EUR, you pay €2500 and receive ฿1  when the order is closed
print(exchange.id, exchange.create_limit_buy_order('BTC/EUR', 1, 2500.00))

# pass/redefine custom exchange-specific order params: type, amount, price, flags, etc...
polo.create_market_buy_order('BTC/USD', 1, {'trading_agreement': 'agree'})